In [9]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix

import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

In [17]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("No of GPUs avail: ",len(physical_devices))
# physical_devices = tf.config.experimental.set_memory_growth(physical_devices[0],True)

No of GPUs avail:  0


IndexError: list index out of range

**Data Prepration**

In [33]:

# Step 1: Create folders if they don't exist
folders = ['train/cat', 'train/dog', 'valid/cat', 'valid/dog', 'test/cat', 'test/dog']
for folder in folders:
    os.makedirs(folder, exist_ok=True)

# Step 2: Get image paths
cat_images = [os.path.join('Cat', f) for f in os.listdir('Cat') if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
dog_images = [os.path.join('Dog', f) for f in os.listdir('Dog') if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# Shuffle to randomize
random.shuffle(cat_images)
random.shuffle(dog_images)

# Step 3: Set counts
train_count = 500
valid_count = 100
test_count = 100

# Step 4: Move cat images
for f in cat_images[:train_count]:
    shutil.move(f, 'train/cat')
for f in cat_images[train_count:train_count+valid_count]:
    shutil.move(f, 'valid/cat')
for f in cat_images[train_count+valid_count:train_count+valid_count+test_count]:
    shutil.move(f, 'test/cat')

# Step 5: Move dog images
for f in dog_images[:train_count]:
    shutil.move(f, 'train/dog')
for f in dog_images[train_count:train_count+valid_count]:
    shutil.move(f, 'valid/dog')
for f in dog_images[train_count+valid_count:train_count+valid_count+test_count]:
    shutil.move(f, 'test/dog'
               )

In [34]:
# import os

# print("train/cat:", len(os.listdir('data/train/cat')))
# print("train/dog:", len(os.listdir('data/train/dog')))
# print("valid/cat:", len(os.listdir('data/valid/cat')))
# print("valid/dog:", len(os.listdir('data/valid/dog')))
# print("test/cat:", len(os.listdir('data/test/cat')))
# print("test/dog:", len(os.listdir('data/test/dog')))


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data/train/cat'

In [35]:
import os
print("Current directory:", os.getcwd())


Current directory: C:\Users\babal\Desktop\MachineLearning\Keras\data


In [36]:
train_path = 'train'
valid_path = 'valid'
test_path = 'test'

In [37]:
train_batches = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input
).flow_from_directory(
    directory=
    train_path,                  # or use `train_path` if defined
    target_size=(224, 224),             # fixed the typo here
    classes=['cat', 'dog'],
    batch_size=10 ,
     shuffle=False 
    # fixed typo: `batchsize` → `batch_size`
)

test_batches = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input
).flow_from_directory(
    directory=test_path,                  # or use `train_path` if defined
    target_size=(224, 224),             # fixed the typo here
    classes=['cat', 'dog'],
    batch_size=10                       # fixed typo: `batchsize` → `batch_size`
)

valid_batches = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input
).flow_from_directory(
    directory=valid_path,                  # or use `train_path` if defined
    target_size=(224, 224),             # fixed the typo here
    classes=['cat', 'dog'],
    batch_size=10                       # fixed typo: `batchsize` → `batch_size`
)



Found 2000 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
